In [56]:
import zipfile
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
print (session.run(c))

[[22. 28.]
 [49. 64.]]


In [2]:
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
'''Create the model before Dense layer'''

def create_model():
    inputs = Input(shape=(144,192,3))

    # Block 1
    block_1_conv_1 = Conv2D(64, kernel_size=(3,3), strides=1, padding='same')(inputs)
    block_1_conv_1 = BatchNormalization()(block_1_conv_1)
    block_1_conv_1 = Activation('relu')(block_1_conv_1)

    block_1_conv_2 = Conv2D(64, kernel_size=(3,3), strides=1, padding='same')(block_1_conv_1)
    block_1_conv_2 = BatchNormalization()(block_1_conv_2)
    block_1_conv_2 = Activation('relu')(block_1_conv_2)

    block_1_pool = MaxPooling2D(pool_size=(2,2))(block_1_conv_2)

    # Block 2
    block_2_conv_1 = Conv2D(128, kernel_size=(3,3), strides=1, padding='same')(block_1_pool)
    block_2_conv_1 = BatchNormalization()(block_2_conv_1)
    block_2_conv_1 = Activation('relu')(block_2_conv_1)

    block_2_conv_2 = Conv2D(128, kernel_size=(3,3), strides=1, padding='same')(block_2_conv_1)
    block_2_conv_2 = BatchNormalization()(block_2_conv_2)
    block_2_conv_2 = Activation('relu')(block_2_conv_2)

    block_2_pool = MaxPooling2D(pool_size=(2,2))(block_2_conv_2)

    # Block 3 (Concatenate block 1 and block 3)
    block_1_block_3 = Cropping2D(((18,18),(24,24)))(block_1_pool)
    concat_block_3 = concatenate([block_1_block_3, block_2_pool])

    block_3_conv_1 = Conv2D(256, kernel_size=(3,3), strides=1, padding='same')(concat_block_3)
    block_3_conv_1 = BatchNormalization()(block_3_conv_1)
    block_3_conv_1 = Activation('relu')(block_3_conv_1)

    block_3_conv_2 = Conv2D(256, kernel_size=(3,3), strides=1, padding='same')(block_3_conv_1)
    block_3_conv_2 = BatchNormalization()(block_3_conv_2)
    block_3_conv_2 = Activation('relu')(block_3_conv_2)

    block_3_conv_3 = Conv2D(256, kernel_size=(3,3), strides=1, padding='same')(block_3_conv_2)
    block_3_conv_3 = BatchNormalization()(block_3_conv_3)
    block_3_conv_3 = Activation('relu')(block_3_conv_3)

    block_3_pool = MaxPooling2D(pool_size=(2,2))(block_3_conv_3)

    # Block 4 Concatenate block1, block 2 and block 4
    block_1_block_4 = Cropping2D(((27,27),(36,36)))(block_1_pool)
    block_2_block_4 = Cropping2D(((9,9),(12,12)))(block_2_pool)
    concat_block_4 = concatenate([block_1_block_4, block_2_block_4, block_3_pool])

    outputs = GlobalAveragePooling2D()(concat_block_4)
        outputs = Dense(60)(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Activation('softmax')(outputs)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    
    return model
#     model.summary()

In [4]:
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def get_paths(directory):
    paths = []
    labels = []
    brands = os.listdir(directory)
    
    for b in brands:
        temp = os.listdir(os.path.join(directory, b))
        temp = [os.path.join(directory, b, k) for k in temp]
        temp_labels = np.repeat(b,len(temp))
        paths.append(temp)
        labels.append(temp_labels)
        
                      
    paths = np.array(paths)
    paths = np.concatenate(paths)
                      
    labels = np.array(labels)
    labels = np.concatenate(labels)
    
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    
    encoded_labels = np.zeros((len(integer_encoded), 51))
    for i, label in enumerate(integer_encoded):
        encoded_labels[i, label] = 1             
    
    return paths, labels, encoded_labels

train_paths, train_labels, encoded_train_labels = get_paths('./dataset/train_data_vietnam_blue_2/')
validation_paths, validation_labels, encoded_validation_labels = get_paths('./dataset/validation_data_vietnam_blue_2/')
test_paths, test_labels, encoded_test_labels = get_paths('./dataset/test_data_vietnam_blue_2/')

img_paths = np.concatenate((train_paths, validation_paths))
img_labels = np.concatenate((train_labels, validation_labels))
img_encoded_labels = np.concatenate((encoded_train_labels, encoded_validation_labels))

img_paths.shape, img_labels.shape, img_encoded_labels.shape, test_paths.shape, encoded_test_labels.shape

((5839,), (5839,), (5839, 51), (1945,), (1945, 51))

In [ ]:

img = imread(img_paths[0])
t = resize(img, (144,192))
t.shape

In [5]:
def load_image_by_indices(indices, img_paths, encoded_label_paths):
    imgs = [img_paths[i] for i in indices]
    encoded_labels = [encoded_label_paths[i] for i in indices]
    
    imgs = np.array(imgs)
    encoded_labels = np.array(encoded_labels)
    return imgs, encoded_labels

In [6]:
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import StratifiedKFold
import time

skf = StratifiedKFold(n_splits=5, shuffle=True)
counter = 0

training_data = []

for train_index, validation_index in skf.split(img_paths, img_labels):
    print('Fold ', counter, '**************************')
    print("TRAIN:", len(train_index), "VALIDATION:", len(validation_index))
    
    # get images paths and their labels
    train_paths, train_labels = load_image_by_indices(train_index, img_paths, img_encoded_labels)
    validation_paths, validation_labels = load_image_by_indices(validation_index, img_paths, img_encoded_labels)
    
    # read images path
#     train_images = np.zeros((len(train_paths), 144,192,3))
#     for i, path in enumerate(train_paths):
#         train_images[i,:,:,:] = resize(imread(path), (144,192))
        
#     validation_images = np.zeros((len(validation_paths), 144,192,3))
#     for i, path in enumerate(validation_paths):
#         validation_images[i,:,:,:] = resize(imread(path), (144,192))
    
    
    training_data.append({
        'train_set': [train_paths, train_labels],
        'validation_set': [validation_paths, validation_labels]
    })
    
    counter += 1
    

('Fold ', 0, '**************************')
('TRAIN:', 4649, 'VALIDATION:', 1190)
('Fold ', 1, '**************************')
('TRAIN:', 4662, 'VALIDATION:', 1177)
('Fold ', 2, '**************************')
('TRAIN:', 4672, 'VALIDATION:', 1167)
('Fold ', 3, '**************************')
('TRAIN:', 4682, 'VALIDATION:', 1157)
('Fold ', 4, '**************************')
('TRAIN:', 4691, 'VALIDATION:', 1148)


In [ ]:
import pickle
with open('/data/Quan/cap_bottle/models/blue/cross_validation/k_fold_data.hdf5', 'wb') as dt:
    pickle.dump(training_data, dt)

In [10]:
import os
import time
import pickle

def cross_validate():
    execution_time = []
    results = []
    batch_size = 64

    train_datagen = ImageDataGenerator(rotation_range=360,
                                        rescale=1./255,
                                        fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    x_test = [imread(path) for path in test_paths]
    x_test = np.array(x_test)
    y_test = encoded_test_labels
    test_generator = test_datagen.flow(x_test, y_test, batch_size=1, shuffle=False)
    
    for i in range(len(training_data)):
        
        test_generator.reset()
        
        model = create_model()

        x_train = np.zeros((len(training_data[i]['train_set'][0]), 144, 192, 3))
        for index, path in enumerate(training_data[i]['train_set'][0]):
            x_train[index,:,:,:] = resize(imread(path), (144,192))
        y_train = training_data[i]['train_set'][1]
        
        x_validation = np.zeros((len(training_data[i]['validation_set'][0]), 144, 192, 3))
        for index, path in enumerate(training_data[i]['validation_set'][0]):
            x_validation[index,:,:,:] = resize(imread(path), (144,192))
        y_validation = training_data[i]['validation_set'][1]

        train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True)
        validation_generator = test_datagen.flow(x_validation, y_validation, batch_size=batch_size, shuffle=False)
        
        
        training_path = '/data/Quan/cap_bottle/models/blue/cross_validation/model_' + str(i) + '.hdf5'
        mcp = ModelCheckpoint(training_path, monitor='val_acc', save_best_only=True, save_weights_only=False)
        
        start = time.time()
        history = model.fit_generator(train_generator, steps_per_epoch=len(x_train)//batch_size, 
                                epochs=260, verbose=1, validation_data=validation_generator, 
                                validation_steps=len(x_validation)//batch_size, callbacks=[mcp])
        
        exe_time = time.time() - start
        
        test_model = load_model(training_path)
        validation_score = test_model.evaluate_generator(validation_generator, 
                                              steps=len(x_validation)//batch_size,
                                              verbose=1)
        test_score = test_model.evaluate_generator(test_generator, steps=len(x_test))
        print('index: ', i, ' | validation_score: ', validation_score, 
              ' | test_score: ', test_score, ' | time: ', exe_time)
        
        # save model
        results.append({
            'index': i,
            'history': history.history,
            'time': exe_time,
            'validation_score': validation_score,
            'test_score': test_score
        })
        
        with open('/data/Quan/cap_bottle/models/blue/cross_validation/results.hdf5', 'wb') as dt:
            pickle.dump(results, dt)
        
        K.clear_sessions()
        
        

### CAM for robust classification

In [27]:
from skimage.io import imread
from skimage.transform import resize
import os
import numpy as np
import matplotlib.pyplot as plt

paths = os.listdir('./good_classification')
paths = [os.path.join('./good_classification', k) for k in paths]

# images = [imread(k) for k in paths]
# images = np.asarray(images)
# images.shape

In [5]:
img = imread(paths[0])
img.shape

(240, 320, 3)

In [ ]:
images = np.empty((27,144,192,3))

for i, p in enumerate(paths):
    images[i,:,:,:] = resize(imread(p),(144,192))

In [28]:
import cv2
def visualize_cam(model, image, last_conv_layer_index, learning_phase=0, show=True, path_to_save=None):
    '''
  visualize class activation map function
  ----------------------------------------
  arguments:
  - model: CNN model with average pooling layer
  - image: image to apply cam
  - last_conv_layer_index: index of last convolution layer
  - learning_phase: integer, if 1 then mode='learing', else if 0 then mode='testing'
  - show: boolean, True then show image using matplotlib
  - path_to_save: path to save the image
  -----------------------------------------
  return:
    predictions: softmax vector
  '''

    '''Get weights of dense output layer'''
    class_weights = model.layers[-3].get_weights()[0]

    '''Create the function to get last conv layer output and model output'''
    last_conv_layer = model.layers[last_conv_layer_index].output
    get_output = K.function([model.input, K.learning_phase()], [last_conv_layer, model.output])

    img = np.array([np.transpose(np.float32(image), (0, 1, 2))])

    [conv_outputs, predictions] = get_output([img, learning_phase])
    conv_outputs = conv_outputs[0,:,:,:]

    '''Create the class activation map'''
    class_num = np.argmax(predictions)
    cam = np.zeros(dtype=np.float32, shape=conv_outputs.shape[:2])

    for i,w in enumerate(class_weights[:,class_num]):
        cam += w*conv_outputs[:,:,i]
        cam /= np.max(cam)
        cam = cv2.resize(cam, (image.shape[1], image.shape[0]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.2)] = 0
        in_heatmap = 255 - heatmap

        plt.imshow(image)
        plt.imshow(in_heatmap, alpha=0.6)
        plt.axis('off')
        if not path_to_save is None:
            plt.savefig(path_to_save, dpi=100)
            if show == True:
                plt.show()

                #   return predictions
                return in_heatmap



In [29]:
model = load_model('./cam_model_ver2_4.hdf5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 144, 192, 3)  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 144, 192, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 144, 192, 64) 256         conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 144, 192, 64) 0           batch_normalization_17[0][0]     
__________________________________________________________________________________________________
conv2d_16 

In [ ]:
%matplotlib inline
from skimage import io

for p in paths[0]:
    image = io.resize(imread(p),(144,192))
    visualize_cam(model, image, last_conv_layer_index=-5, learning_phase=0, show=True, path_to_save='./cam_good_classification')

In [30]:
img = resize(imread(paths[1]),(144,192))

In [31]:
img.shape

(144, 192, 3)

In [ ]:
visualize_cam(model, img, last_conv_layer_index=-5, learning_phase=0, show=True, path_to_save='./cam_good_classification/1.png')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in less


In [3]:
model = load_model('./cam_model_ver2_4.hdf5')
model.summary()

W1129 03:49:02.117606 140528055035648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1129 03:49:02.123076 140528055035648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1129 03:49:02.143611 140528055035648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1129 03:49:02.144120 140528055035648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1129 03:49:02.144597 140528055

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 144, 192, 3)  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 144, 192, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 144, 192, 64) 256         conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 144, 192, 64) 0           batch_normalization_17[0][0]     
__________________________________________________________________________________________________
conv2d_16 

In [40]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

test_generator = datagen.flow_from_directory('./dataset/test_data_vietnam_blue_2', target_size=(144,192),
                                                batch_size=1,
                                                 shuffle=False,
                                                class_mode='binary')

Found 1945 images belonging to 51 classes.


In [41]:
predictions = model.predict_generator(test_generator, steps=len(test_generator.filenames), verbose=1)
predictions.shape

1945/1945 [==============================] - 6s 3ms/step


(1945, 51)

In [20]:
true_labels = [k.split('/')[0] for k in test_generator.filenames]
true_labels

['Brasserie de Saint-Omer devissez',
 'Brasserie de Saint-Omer devissez',
 'Brasserie de Saint-Omer devissez',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'affligem-blonde',
 'affligem-blonde',
 'affligem-blonde',
 'almaza',
 'almaza',
 'almaza',
 'almaza',
 'almaza',
 'almaza-old',
 'almaza-old',
 'almaza-old',
 'almaza-old',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s

In [49]:
labels_dict = [(0, 'Brasserie de Saint-Omer devissez'),
 (1, 'Schmucker-hefe-weizen'),
 (2, 'affligem-blonde'),
 (3, 'almaza'),
 (4, 'almaza-old'),
 (5, 'beck_s-bier'),
 (6, 'beck_s-ice'),
 (7, 'beerlao'),
 (8, 'bernard-pale-lager'),
 (9, 'bia-quy-nhon'),
 (10, 'budweiser'),
 (11, 'carlsberg'),
 (12, 'chang'),
 (13, 'coopers-sparkling-ale'),
 (14, 'corona-extra'),
 (15, 'desperados'),
 (16, 'duvel'),
 (17, 'gau-den'),
 (18, 'hb-muenchner-weisse'),
 (19, 'heineken'),
 (20, 'hoegaarden-anno-1445'),
 (21, 'hoegaarden-rosee'),
 (22, 'hoegaarden-simple'),
 (23, 'hoegaarden-wit-blanche'),
 (24, 'huda'),
 (25, 'leffe-blonde'),
 (26, 'leffe-brown'),
 (27, 'levrette-cherry'),
 (28, 'lowen'),
 (29, 'oberbrau-1605-hefe-weisse'),
 (30, 'orion-premium-draft-beer'),
 (31, 'peroni nastro azzurro'),
 (32, 'pilsner-urquel-original'),
 (33, 'rogue-ales'),
 (34, 'sagota-larger'),
 (35, 'saigon-export'),
 (36, 'saigon-lager'),
 (37, 'saigon-special'),
 (38, 'san-miguel'),
 (39, 'san-miguel-light'),
 (40, 'sapporo'),
 (41, 'shingha'),
 (42, 'spaten-optimator'),
 (43, 'staropramen-premium'),
 (44, 'stella-artois'),
 (45, 'strongbow-elderflower'),
 (46, 'strongbow-goldapple'),
 (47, 'strongbow-honey'),
 (48, 'strongbow-redberries'),
 (49, 'tiger'),
 (50, 'trio-extra-stout')]

labels_dict = np.asarray(labels_dict)

In [42]:
preds = [np.argmax(k, axis=0) for k in predictions]
preds = np.asarray(preds)
preds.shape

(1945,)

In [43]:
preds

array([ 0,  0,  0, ..., 50, 50, 50])

In [51]:
pred_labels = [labels_dict[k] for k in preds]
len(pred_labels)

1945

In [52]:
pred_labels = [k[1] for k in pred_labels]
pred_labels

['Brasserie de Saint-Omer devissez',
 'Brasserie de Saint-Omer devissez',
 'Brasserie de Saint-Omer devissez',
 'desperados',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'Schmucker-hefe-weizen',
 'affligem-blonde',
 'affligem-blonde',
 'affligem-blonde',
 'almaza',
 'almaza',
 'almaza',
 'almaza',
 'almaza',
 'almaza-old',
 'almaza-old',
 'almaza-old',
 'almaza-old',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'beck_s-bier',
 'b

In [59]:
!pip install scikit-learn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pickle 

with open('./y_true.hdf5', 'rb') as dt:
    y_true = pickle.load(dt)
    
with open('./y_pred.hdf5', 'rb') as dt:
    y_pred = pickle.load(dt)

In [3]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

                                  precision    recall  f1-score   support

Brasserie de Saint-Omer devissez       1.00      1.00      1.00         3
           Schmucker-hefe-weizen       1.00      0.80      0.89         5
                 affligem-blonde       1.00      1.00      1.00         3
                          almaza       1.00      1.00      1.00         5
                      almaza-old       1.00      1.00      1.00         4
                     beck_s-bier       0.98      1.00      0.99        42
                      beck_s-ice       1.00      1.00      1.00        46
                         beerlao       1.00      1.00      1.00         4
              bernard-pale-lager       1.00      1.00      1.00         3
                    bia-quy-nhon       0.86      1.00      0.92         6
                       budweiser       1.00      1.00      1.00       249
                       carlsberg       1.00      0.95      0.98        22
                           chang     

In [61]:
import pickle 

with open('./y_true.hdf5', 'wb') as dt:
    pickle.dump(true_labels, dt)
    
with open('./y_pred.hdf5', 'wb') as dt:
    pickle.dump(pred_labels, dt)